In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
import pyspark
print(pyspark.__version__)
!java -version

3.5.0
java version "21.0.2" 2024-01-16 LTS
Java(TM) SE Runtime Environment (build 21.0.2+13-LTS-58)
Java HotSpot(TM) 64-Bit Server VM (build 21.0.2+13-LTS-58, mixed mode, sharing)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
         .appName("SparkSQL")\
         .getOrCreate()

24/01/25 13:33:36 WARN Utils: Your hostname, Andrews-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.202 instead (on interface en0)
24/01/25 13:33:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/25 13:33:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

home_sales_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv(SparkFiles.get('home_sales_revised.csv'))

In [5]:
# Preview dataframe
home_sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [6]:
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)


In [7]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createTempView('home_sales')

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import year
from pyspark.sql.functions import round, avg

home_sales_df = home_sales_df.withColumn("Year", year(home_sales_df.date))

home_sales_df.filter(home_sales_df.bedrooms == 4)\
    .select(['bedrooms', year(home_sales_df.date).alias('Year'), 'price']) \
    .groupBy('Year', 'bedrooms')\
    .agg(round(avg('price'), 2).alias('AVG_PRICE'))\
    .orderBy('AVG_PRICE')\
    .show()


+----+--------+---------+
|Year|bedrooms|AVG_PRICE|
+----+--------+---------+
|2022|       4|296363.88|
|2020|       4|298353.78|
|2019|       4| 300263.7|
|2021|       4|301819.44|
+----+--------+---------+


In [9]:
# 3b. PySpark with SQL
query = '''SELECT bedrooms, YEAR(date) as YEAR, ROUND(AVG(price), 2) as AVG_PRICE 
    FROM home_sales 
    WHERE bedrooms = 4 
    GROUP BY YEAR, bedrooms'''

spark.sql(query).show(truncate=False)

+--------+----+---------+
|bedrooms|YEAR|AVG_PRICE|
+--------+----+---------+
|4       |2020|298353.78|
|4       |2019|300263.7 |
|4       |2022|296363.88|
|4       |2021|301819.44|
+--------+----+---------+


In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
home_sales_df.filter((home_sales_df.bedrooms == 3) & (home_sales_df.bathrooms == 3)) \
    .groupBy('date_built') \
    .agg(round(avg("price"), 2).alias("AVG_PRICE")) \
    .orderBy('AVG_PRICE') \
    .show()

+----------+---------+
|date_built|AVG_PRICE|
+----------+---------+
|      2015| 288770.3|
|      2016|290555.07|
|      2014|290852.27|
|      2011|291117.47|
|      2017|292676.79|
|      2010|292859.62|
|      2012|293683.19|
|      2013|295962.27|
+----------+---------+


In [11]:
# 4b
query = '''SELECT date_built, ROUND(AVG(price), 2) as AVG_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built 
'''

spark.sql(query).show(truncate=False)

+----------+---------+
|date_built|AVG_PRICE|
+----------+---------+
|2015      |288770.3 |
|2013      |295962.27|
|2014      |290852.27|
|2012      |293683.19|
|2016      |290555.07|
|2010      |292859.62|
|2011      |291117.47|
|2017      |292676.79|
+----------+---------+


In [12]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

home_sales_df.filter((home_sales_df.bedrooms == 3) & 
                     (home_sales_df.bathrooms == 3) & 
                     (home_sales_df.floors == 2) & 
                     (home_sales_df.sqft_living >= 2000))\
    .select(year(home_sales_df.date).alias('YEAR'), 
            'price')\
    .groupBy('YEAR')\
    .agg(round(avg('price'), 2).alias('AVG_PRICE'))\
    .show()


+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2022|290242.99|
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
+----+---------+


In [13]:
#5b
query = ''' SELECT YEAR(date) as YEAR, ROUND(AVG(PRICE), 2) as AVG_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY YEAR
'''

spark.sql(query).show(truncate=False)

+----+---------+
|YEAR|AVG_PRICE|
+----+---------+
|2022|290242.99|
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
+----+---------+


In [14]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

home_sales_df.filter((home_sales_df.price >= 350_000))\
    .select('price', 'view')\
    .groupBy('view')\
    .agg(round(avg('price'), 2).alias('AVG_PRICE'))\
    .orderBy('view')\
    .show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|AVG_PRICE|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 0.1961808204650879 seconds ---


In [15]:
# 6b
start_time = time.time()
query = ''' SELECT ROUND(AVG(price),2) AS AVG_PRICE, view
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC
'''

spark.sql(query).show()
print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| AVG_PRICE|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.15936517715454102 seconds ---


In [19]:
# 7. Cache the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()
query = """
SELECT view, ROUND(AVG(price), 2) AS AVG_PRICE
FROM home_sales
GROUP BY view
HAVING AVG_PRICE >= 350000
ORDER BY view DESC
"""
spark.sql(query).show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|AVG_PRICE |
+----+----------+
|100 |1026669.5 |
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
+----+----------+
only showing top 20 rows

--- 0.11849379539489746 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.partitionBy('date_built')\
    .parquet('home_sales_parquet', mode='overwrite')

In [23]:
# 11. Read the formatted parquet data.
home_sales_p_df = spark.read.parquet('home_sales_parquet')

In [26]:
# 12. Create a temporary table for the parquet data.
home_sales_p_df.createTempView('home_sales_parq')

In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
query = '''
SELECT view, ROUND(AVG(price),2) AS AVG_PRICE
FROM home_sales_parq
GROUP BY view
HAVING AVG_PRICE >= 350000
ORDER BY view DESC
'''
spark.sql(query).show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|AVG_PRICE |
+----+----------+
|100 |1026669.5 |
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
+----+----------+
only showing top 20 rows

--- 0.33395814895629883 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales_parq')

DataFrame[]

In [29]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales_parq')

False

In [30]:
# close session
spark.stop()